# Utilising the dummy data for each city

In [26]:
import pandas as pd
import geopandas as gpd
import json
import os
import sys
# go up one level from notebooks/ and into src/
sys.path.append(os.path.abspath("../src"))
import schema

In [27]:
json_path = "../data/Location/reallocate_all_pilots.geojson"
with open(json_path, "r") as f:
    pilot_geometry = gpd.read_file(f)

#Fix pilot names
pilot_geometry["pilot"] = pilot_geometry["city"] + "_" + pilot_geometry["pilot"].str.lstrip("Pilot")
pilot_geometry.head()

# If a pilot has multiple geometries, dissolve them into one
pilot_geometry = pilot_geometry.dissolve(by="pilot", as_index=False)
print("The shape of the pilot geometry is:", pilot_geometry.shape)
pilot_geometry["pilot"].unique()

The shape of the pilot geometry is: (7, 4)


array(['Barcelona_1', 'Budapest_1', 'Budapest_2', 'Gothenburg_2',
       'Heidelberg_1', 'Utrecht_1', 'Utrecht_2'], dtype=object)

In [58]:
csv_path = "../data/Location/pilot_static_data.csv"
with open(csv_path, "r") as f:
    pilot_coordinates = pd.read_csv(f,sep="	")

print(pilot_coordinates.shape)

pilot_coordinates["lat"] = pilot_coordinates["lat"].astype(str).str.replace(",", ".")
pilot_coordinates["lon"] = pilot_coordinates["lon"].astype(str).str.replace(",", ".")

pilot_coordinates["geometry"] = gpd.points_from_xy(pilot_coordinates.lon, pilot_coordinates.lat)
pilot_coordinates = gpd.GeoDataFrame(pilot_coordinates, geometry="geometry", crs="EPSG:4326")
print(pilot_coordinates.shape)
pilot_coordinates = pilot_coordinates[["City","geometry"]].copy()
pilot_coordinates.head()


(15, 21)
(15, 22)


,City,geometry
0,Gothenburg_1,POINT (11.97456 57.70887)
1,Gothenburg_2,POINT (11.99456 57.72887)
2,Heidelberg_1,POINT (8.67243 49.39875)
3,Heidelberg_2,POINT (8.65243 49.37875)
4,Barcelona_1,POINT (2.18138 41.38878)


In [7]:
json_path = "../data/Dummy/pilot_kpis.json"
with open(json_path, "r") as f:
    pilot_data = json.load(f)

# Flatten the nested structure
rows = []
for pilot, kpis in pilot_data.items():
    for kpi in kpis:
        rows.append({
            "pilot": pilot,
            **kpi  # expands all KPI keys
        })

df_json = pd.DataFrame(rows)
df_json



,pilot,name,theme,kpi_theme_id,unit,baseline_value,post_value
0,Barcelona_1,Number of participants engaged,Engagement,1,counts,57.45,143.59
1,Barcelona_1,reported risk factors affecting perceived road...,road safety,2,counts,93.20,97.83
2,Barcelona_2,Engagement in co-creation,Engagement,1,counts,35.60,40.28
3,Barcelona_2,Increase in users of special services through DRT,Accessibility,3,counts,25.81,132.60
4,Heidelberg_1,Engagement in co-creation,Engagement,1,counts,80.11,112.05
5,Heidelberg_1,Collaboration agreement,Engagement,1,counts,22.06,146.09
6,Heidelberg_2,Pedestrian and Cyclist counts,road safety,2,counts,103.24,47.60
7,Heidelberg_2,Impact on traffic flows,road safety,2,Vehicles/24h,1181.82,1183.40
8,Heidelberg_2,Conversion from impermeable to permeable/veget...,Environment,4,%,30.42,52.48
9,Gothenburg_1,Engagement in co-creation,Engagement,1,Counts,85.92,86.21


In [8]:
model = schema.AllCitiesKPIs.model_validate(pilot_data)
print(model.get_city("Barcelona_1"))

[KPI(name='Number of participants engaged', theme='Engagement', kpi_theme_id='1', unit='counts', baseline_value=57.45, post_value=143.59), KPI(name='reported risk factors affecting perceived road safety', theme='road safety', kpi_theme_id='2', unit='counts', baseline_value=93.2, post_value=97.83)]


In [9]:
model.get_city("Barcelona_1")

[KPI(name='Number of participants engaged', theme='Engagement', kpi_theme_id='1', unit='counts', baseline_value=57.45, post_value=143.59),
 KPI(name='reported risk factors affecting perceived road safety', theme='road safety', kpi_theme_id='2', unit='counts', baseline_value=93.2, post_value=97.83)]

# Explore Join

In [61]:
join_results = df_json.merge(pilot_geometry, on="pilot", how="left")
print("The original shape is:", df_json.shape, "and the joined shape is:", join_results.shape)

# If a pilot has no geometry, merge with the coordinates dataset
join_results = join_results.merge(pilot_coordinates, left_on="pilot", right_on="City", how="left", suffixes=("", "_coord"))
print("The shape after merging with coordinates is:", join_results.shape)

# If geometry is missing, use the coordinates geometry
join_results["geometry"] = join_results["geometry"].combine_first(join_results["geometry_coord"])
join_results = join_results.drop(columns=["City", "geometry_coord"])
print("The final shape is:", join_results.shape)
join_results.head()

The original shape is: (30, 7) and the joined shape is: (30, 10)
The shape after merging with coordinates is: (30, 12)
The final shape is: (30, 10)


,pilot,name,theme,kpi_theme_id,unit,baseline_value,post_value,geometry,id,city
0,Barcelona_1,Number of participants engaged,Engagement,1,counts,57.45,143.59,"MULTIPOLYGON (((2.1232 41.37588, 2.14844 41.37...",0,Barcelona
1,Barcelona_1,reported risk factors affecting perceived road...,road safety,2,counts,93.20,97.83,"MULTIPOLYGON (((2.1232 41.37588, 2.14844 41.37...",0,Barcelona
2,Barcelona_2,Engagement in co-creation,Engagement,1,counts,35.60,40.28,POINT (2.16138 41.36878),NaN,NaN
3,Barcelona_2,Increase in users of special services through DRT,Accessibility,3,counts,25.81,132.60,POINT (2.16138 41.36878),NaN,NaN
4,Heidelberg_1,Engagement in co-creation,Engagement,1,counts,80.11,112.05,"MULTIPOLYGON (((8.69101 49.41531, 8.69138 49.4...",5,Heidelberg


In [60]:
join_results

,pilot,name,theme,kpi_theme_id,unit,baseline_value,post_value,geometry,id,city,City,geometry_coord
0,Barcelona_1,Number of participants engaged,Engagement,1,counts,57.45,143.59,"MULTIPOLYGON (((2.1232 41.37588, 2.14844 41.37...",0,Barcelona,Barcelona_1,POINT (2.18138 41.38878)
1,Barcelona_1,reported risk factors affecting perceived road...,road safety,2,counts,93.20,97.83,"MULTIPOLYGON (((2.1232 41.37588, 2.14844 41.37...",0,Barcelona,Barcelona_1,POINT (2.18138 41.38878)
2,Barcelona_2,Engagement in co-creation,Engagement,1,counts,35.60,40.28,None,NaN,NaN,Barcelona_2,POINT (2.16138 41.36878)
3,Barcelona_2,Increase in users of special services through DRT,Accessibility,3,counts,25.81,132.60,None,NaN,NaN,Barcelona_2,POINT (2.16138 41.36878)
4,Heidelberg_1,Engagement in co-creation,Engagement,1,counts,80.11,112.05,"MULTIPOLYGON (((8.69101 49.41531, 8.69138 49.4...",5,Heidelberg,Heidelberg_1,POINT (8.67243 49.39875)
5,Heidelberg_1,Collaboration agreement,Engagement,1,counts,22.06,146.09,"MULTIPOLYGON (((8.69101 49.41531, 8.69138 49.4...",5,Heidelberg,Heidelberg_1,POINT (8.67243 49.39875)
6,Heidelberg_2,Pedestrian and Cyclist counts,road safety,2,counts,103.24,47.60,None,NaN,NaN,Heidelberg_2,POINT (8.65243 49.37875)
7,Heidelberg_2,Impact on traffic flows,road safety,2,Vehicles/24h,1181.82,1183.40,None,NaN,NaN,Heidelberg_2,POINT (8.65243 49.37875)
8,Heidelberg_2,Conversion from impermeable to permeable/veget...,Environment,4,%,30.42,52.48,None,NaN,NaN,Heidelberg_2,POINT (8.65243 49.37875)
9,Gothenburg_1,Engagement in co-creation,Engagement,1,Counts,85.92,86.21,None,NaN,NaN,Gothenburg_1,POINT (11.97456 57.70887)


In [24]:
join_results

,pilot,name,theme,kpi_theme_id,unit,baseline_value,post_value,geometry,id,city
0,Barcelona_1,Number of participants engaged,Engagement,1,counts,57.45,143.59,"MULTIPOLYGON (((2.1232 41.37588, 2.14844 41.37...",0,Barcelona
1,Barcelona_1,reported risk factors affecting perceived road...,road safety,2,counts,93.20,97.83,"MULTIPOLYGON (((2.1232 41.37588, 2.14844 41.37...",0,Barcelona
2,Barcelona_2,Engagement in co-creation,Engagement,1,counts,35.60,40.28,None,NaN,NaN
3,Barcelona_2,Increase in users of special services through DRT,Accessibility,3,counts,25.81,132.60,None,NaN,NaN
4,Heidelberg_1,Engagement in co-creation,Engagement,1,counts,80.11,112.05,"MULTIPOLYGON (((8.69101 49.41531, 8.69138 49.4...",5,Heidelberg
5,Heidelberg_1,Collaboration agreement,Engagement,1,counts,22.06,146.09,"MULTIPOLYGON (((8.69101 49.41531, 8.69138 49.4...",5,Heidelberg
6,Heidelberg_2,Pedestrian and Cyclist counts,road safety,2,counts,103.24,47.60,None,NaN,NaN
7,Heidelberg_2,Impact on traffic flows,road safety,2,Vehicles/24h,1181.82,1183.40,None,NaN,NaN
8,Heidelberg_2,Conversion from impermeable to permeable/veget...,Environment,4,%,30.42,52.48,None,NaN,NaN
9,Gothenburg_1,Engagement in co-creation,Engagement,1,Counts,85.92,86.21,None,NaN,NaN
